In [1]:
data_var = '2023-12-15'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
9189,2023-12-15,Japão B.League,07:05,Brave Thunders,Hokkaido,1.07,7.23,156.5,1.84,1.86,-15.5,1.98,1.74,KAWsxylI,0.934579,0.138313,0.543478,0.537634,0.072892,149.280,40.908797,0.274041,1.2,1.643168,1.369306,139.12,1.738,0.307847,0.177127,5.0,262.208,65.946271,0.251504,0.0,0.000000,NaN,246.01,3.434,0.836588,0.243619,-54.0,74,73,1.88,3.37,0.000,212.666,1.049585,0.007644,0.091240,0.00,0.000,inf,0.0,0.0,0.0,0.30,0.060,103.833333,0.0,0.0,0.0
9190,2023-12-15,Irã Superliga,09:30,Tabiat,Zob Ahan,1.61,2.22,158.5,1.85,1.85,-4.5,2.09,1.66,KvrJ9iqb,0.621118,0.450450,0.540541,0.540541,0.071568,130.940,62.659365,0.478535,1.8,1.643168,0.912871,74.12,1.566,0.776582,0.495902,41.0,117.724,25.962348,0.220536,3.0,0.000000,0.000000,122.46,1.484,0.325853,0.219577,52.0,68,78,1.09,1.57,0.000,0.000,0.225240,0.000000,0.162163,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
9191,2023-12-15,Islândia Premier League,16:15,Keflavik,Thorl,1.42,2.72,182.5,1.80,1.86,-6.5,2.03,1.70,dnpfJM9B,0.704225,0.367647,0.555556,0.537634,0.071872,195.974,75.342318,0.384451,2.4,1.341641,0.559017,269.86,1.992,0.625076,0.313793,75.0,152.976,41.517819,0.271401,1.8,1.643168,0.912871,96.32,1.684,0.567389,0.336929,9.0,103,86,2.62,1.12,0.000,0.000,0.444077,0.023184,0.125118,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
9192,2023-12-15,Israel Superliga,09:00,Hapoel Tel-Aviv,Nes Ziona,1.06,8.25,171.5,1.87,1.79,-16.5,1.99,1.73,AHfwBQwH,0.943396,0.121212,0.534759,0.558659,0.064608,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,90,76,1.04,1.40,0.000,0.000,1.092180,0.030912,0.098843,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
9193,2023-12-15,Japão B2.League,07:00,Fukuoka,Ehime Orange Vikings,1.07,7.39,154.5,1.80,1.90,-15.5,1.98,1.74,jJG9RjJR,0.934579,0.135318,0.555556,0.526316,0.069897,114.530,28.197198,0.246199,1.8,1.643168,0.912871,159.31,1.498,0.210404,0.140457,22.0,305.908,77.580289,0.253607,0.0,0.000000,NaN,275.42,3.786,0.932808,0.246383,-75.0,89,94,1.79,2.93,0.000,0.000,1.056481,0.038222,0.091240,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9305,2023-12-15,China Cba,08:35,Nanjing Tongxi,Liaoning,4.11,1.22,200.5,1.86,1.88,8.5,2.07,1.67,0YFJtfkS,0.243309,0.819672,0.537634,0.531915,0.062981,294.180,151.468708,0.514884,1.8,1.643168,0.912871,325.52,2.914,1.676985,0.575493,8.0,122.908,26.618629,0.216574,2.4,1.341641,0.559017,138.99,1.222,0.141845,0.116076,16.0,104,113,3.13,1.23,227.096,137.838,0.766806,0.007563,0.151253,-0.91,-0.182,-17.087912,0.0,0.0,0.0,0.23,0.046,4.782609,0.0,0.0,0.0
9306,2023-12-15,China Cba,08:35,Guangzhou,Tianjin,1.25,3.77,201.5,1.87,1.87,-13.5,2.07,1.67,IkdjyYCk,0.800000,0.265252,0.534759,0.534759,0.065252,171.484,47.691944,0.278113,1.2,1.643168,1.369306,219.78,1.680,0.451276,0.268617,14.0,284.544,125.921865,0.442539,1.2,1.643168,1.369306,156.09,2.686,1.132400,0.421593,-10.0,99,121,2.22,1.29,134.008,634.972,0.709924,0.000000,0.151253,-2.76,-0.552,-0.452899,0.0,0.0,0.0,

## Apostas do dia

### Back Home V1 (QCut)

In [8]:
# Duplicando o dataset
df_bh_v1 = jogos.copy()

df_bh_v1 = df_bh_v1[(df_bh_v1['Odds_H'] >= 1.6) & (df_bh_v1['Odds_H'] <= 2.4)]

if not df_bh_v1.empty:    
    # Filtros pós modelo
    df_bh_v1 = df_bh_v1[((df_bh_v1['CG_A_Last_Game'] < 0.0) | (df_bh_v1['CG_A_Last_Game'] >= 160))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['CV_HA'] < 0.0334) | (df_bh_v1['CV_HA'] >= 1.414))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['P_Diff'] < 0.0464) | (df_bh_v1['P_Diff'] >= 0.065))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['CG_A_Last_Game'] < 348) | (df_bh_v1['CG_A_Last_Game'] >= 920))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['Goals_Last_H'] >= 79) & (df_bh_v1['Goals_Last_H'] <= 110)) | ((df_bh_v1['Goals_Last_H'] > 120.8) & (df_bh_v1['Goals_Last_H'] <= 143))]

    if not df_bh_v1.empty:
        df_bh_v1.to_csv(f'apostas_do_dia/{data_var}/back_home_v1_{data_var}.csv', index=False)

        print('------------ MÉTODO BACK HOME V1 (QCUT) ------------')
        print()

        df_bh_v1['Bet'] = 'Back Home'
        ligs = df_bh_v1['League'].unique().tolist()

        display(df_bh_v1[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO BACK HOME V1 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
9267,21:30,Eua Nba,Toronto Raptors,Atlanta Hawks,1.77,Back Home


### Back Home V2 (LR)

In [9]:
# Duplicando o dataset
df_bh_v2 = jogos.copy()

df_bh_v2 = df_bh_v2[(df_bh_v2['Odds_H'] >= 1.6) & (df_bh_v2['Odds_H'] <= 2.4)]
df_bh_v2.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v2.empty:
    mdl_bh_v2 = load_model('ML/back_home/v2/v2_back_home', verbose=False)
    prev = predict_model(mdl_bh_v2, df_bh_v2)
    prev = prev[prev['prediction_label'] == 1]

    # Filtros pós modelo
    prev = prev[((prev['CV_Odds_Geral_H'] <= 0.224) | (prev['CV_Odds_Geral_H'] > 0.441))]
    prev = prev[((prev['P(U)'] < 0.526) | (prev['P(U)'] > 0.535))]
    prev = prev[((prev['Last_Odd_H'] <= 1.59) | (prev['Last_Odd_H'] > 2.3))]

    if not df_bh_v2.empty:
        df_bh_v2.to_csv(f'apostas_do_dia/{data_var}/back_home_v2_{data_var}.csv', index=False)

        print('------------ MÉTODO BACK HOME V2 (LR) ------------')
        print()

        df_bh_v2['Bet'] = 'Back Home'
        ligs = df_bh_v2['League'].unique().tolist()

        display(df_bh_v2[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO BACK HOME V2 (LR) ------------



,Time,League,Home,Away,Odds_H,Bet
9190,09:30,Irã Superliga,Tabiat,Zob Ahan,1.61,Back Home
9197,13:30,Polônia Liga De Basquete,Lancut,Dabrowa Gornicza,2.20,Back Home
9198,15:00,Polônia 1. Liga,Slask Wroclaw II,Starogard Gdanski,1.69,Back Home
9199,12:00,Geórgia Superliga,Rustavi,Kavkasia,1.86,Back Home
9208,16:00,França Pro B,Chalons-Reims,Boulazac,1.62,Back Home
9210,16:00,França Pro B,Angers,Pau-Orthez,2.22,Back Home
9211,13:30,Finlândia Korisliiga,Lahti Basketball,Bisons Loimaa,2.17,Back Home
9218,15:00,República Tcheca 1. Liga,Litomerice,Plzen,1.66,Back Home
9221,14:00,Finlândia Divisão I Feminina,HBA F,Tapiolan Honka II F,1.72,Back Home
9237,13:00,Turquia Tbl,Bornova,Kocaeli BŞB Kağıtspor,1.78,Back Home


### Under V1 (QCut)

In [10]:
# Duplicando o dataset
df_u_v1 = jogos.copy()

df_u_v1 = df_u_v1[(df_u_v1['Odds_H'] >= 1.6) & (df_u_v1['Odds_H'] <= 2.4)]

if not df_u_v1.empty:    
    # Filtros pós modelo
    df_u_v1 = df_u_v1[((df_u_v1['CG_A_Last_Game'] <= 191) | (df_u_v1['CG_A_Last_Game'] > 280))]
    df_u_v1 = df_u_v1[((df_u_v1['Saldo_Gols_H'] <= 0) | (df_u_v1['Saldo_Gols_H'] > 16))]
    df_u_v1 = df_u_v1[((df_u_v1['CG_H_Last_Game'] <= 166.6) | (df_u_v1['CG_H_Last_Game'] > 260.85))]
    df_u_v1 = df_u_v1[((df_u_v1['Avg_CG_H'] <= 262.034) | (df_u_v1['Avg_CG_H'] > 969.8))]
    df_u_v1 = df_u_v1[((df_u_v1['MediaCustoGolAway'] <= 200.907) | (df_u_v1['MediaCustoGolAway'] > 868.35))]
    df_u_v1 = df_u_v1[((df_u_v1['P_Diff'] <= 0.0747) | (df_u_v1['P_Diff'] > 0.0971))]
    df_u_v1 = df_u_v1[((df_u_v1['HA_Odds_A'] >= 0) & (df_u_v1['HA_Odds_A'] <= 1.68)) | ((df_u_v1['HA_Odds_A'] > 1.8) & (df_u_v1['HA_Odds_A'] <= 1.9))]
    df_u_v1 = df_u_v1[((df_u_v1['CV_HA'] >= 0) & (df_u_v1['CV_HA'] <= 0.162))]
    df_u_v1 = df_u_v1[((df_u_v1['Last_Odd_A'] <= 1.297) | (df_u_v1['Last_Odd_A'] > 1.62))]
    df_u_v1 = df_u_v1[((df_u_v1['P(A)'] <= 0.234) | (df_u_v1['P(A)'] > 0.365))]
    
    if not df_u_v1.empty:
        df_u_v1.to_csv(f'apostas_do_dia/{data_var}/under_v1_{data_var}.csv', index=False)

        print('------------ MÉTODO UNDER V1 (QCUT) ------------')
        print()

        df_u_v1['Bet'] = 'Back Home'
        ligs = df_u_v1['League'].unique().tolist()

        display(df_u_v1[['Time', 'League', 'Home', 'Away', 'Odds_Under', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

Sem jogos


### Back Home V3 (QCut)

In [11]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
9208,16:00,França Pro B,Chalons-Reims,Boulazac,1.62,Back Home
9251,05:30,Austrália Liga Feminina Wnbl,Bendigo F,Adelaide Lightning F,1.54,Back Home
9253,07:30,Austrália Nbl,Perth Wildcats,Tasmania JackJumpers,1.74,Back Home
9256,20:00,Argentina Liga A,Union De Santa Fe,Regatas,1.87,Back Home
